In [1]:
import re
from collections import Counter, defaultdict

import numpy as np

In [2]:
with open("AnnaKarenina.txt", "r", encoding="utf8") as data:
    corpus = data.read()

with open("WarAndPeace.txt", "r", encoding="utf8") as data:
    corpus += data.read()

In [3]:
Regular = re.compile(r"\W+")
corpus = Regular.sub(" ", corpus).lower()
len(corpus)

2385683

In [4]:
train1, test1 = corpus[:-100], corpus[-100:]
train2, test2 = corpus[:-800], corpus[-800:]
train3, test3 = corpus[:-1500], corpus[-1500:]

In [5]:
test1

'ный он не выздоровеет князь андрей в числе других безнадежных раненых был сдан на попечение жителей '

In [6]:
def random_encode_text(text):
    chars = list(set(text))
    mapping = dict(zip(chars, list(map(chr, np.random.choice(range(1, 5000), size=len(chars), replace=False)))))
    encoded = [mapping[char] for char in text]
    return "".join(encoded)

In [7]:
encoded_test1 = random_encode_text(test1)
encoded_test2 = random_encode_text(test2)
encoded_test3 = random_encode_text(test3)

In [8]:
encoded_test1

'\u0e00༓\x99XɈ\u0e00X\u0e00ຶXج༓ᅔӊɈ࠴Ɉجຶຶ\u0b3bXʉ\u0e00ŘᅔƁXମ\u0e00ӊ࠴ຶ\x99XجXض༆\x7f͝ຶXӊ࠴ӻॱ༆\u0558Xܑຶᅔ\u0e00ମӊຶԣ\u0e00༓\u0558X࠴ମ\u0e00ຶ\u0e00༓\u0558Xܑ༓͝X\x7fӊମ\u0e00X\u0e00ମXअɈअຶضຶ\u0e00༆ຶXԣ༆\u0b3bຶ͝ຶ\x99X'

# 1. Частотный метод

Возьмем 3 тестовых предложения и реализуем частотный метод по Шерлоку Холмсу

In [9]:
def unigramm_mapping(encoded, real):
    encoded_count = Counter(encoded).most_common()
    symbol_count = Counter(real)
    mapping = {}
    for position, (symbol, count) in enumerate(symbol_count.most_common(len(encoded_count))):
        mapping[encoded_count[position][0]] = symbol

    return mapping


def decode_text(text, mapping):
    return "".join([mapping[symbol] for symbol in text])


mapping_enc_dec1 = unigramm_mapping(encoded_test1, train1)
mapping_enc_dec2 = unigramm_mapping(encoded_test2, train2)
mapping_enc_dec3 = unigramm_mapping(encoded_test3, train3)
decoded1 = decode_text(encoded_test1, mapping_enc_dec1)
decoded2 = decode_text(encoded_test2, mapping_enc_dec2)
decoded3 = decode_text(encoded_test3, mapping_enc_dec3)

In [10]:
decoded1, decoded2, decoded3

('еав те ео ракнтстрооу безкч иенсов р плядо нсжйлм ьокеиногеам сиеоеам ьад яние еи ытыопоело глуодов ',
 'ияглн кнсоаыьеяе аипвехаеяе анбесоеан и анм рток цвик рутеьео аоче тетвнрсдси яснраео етаернаио ояе яелдпопауж бломтвнрсоаий вижнд хизаы и тбеьейаео токойаео тпнтвио р сутуж яелнж бломтвнрсдситы окг еа гхо антснхмнстд цвик тпнтвиок ьеямн рмлгя дрсдстд кнсоаыьий анбlсоеа т треик чозгпнтваук еялнаипоааук и тпнтвсирук ев аотпнтвид млгяиж рзясдмек и анпианситы текаоаид кгьи и весыье аоче ечоmнсе гтбеьеоаио ь гвлг рто копвнаид ткоaнситы и тсиситы р жнет и клнь чотбнкдвтврн и знчроаид ьевелуо яелнзме роледваоо бе каоаиr тнкеяе снллод меьвелн анбесоеан месхау чуси лнзлоaивытд тколвыr пок рузмелерсоаиок o шщe юэ щюcшe эшфdшюu шe pitiшюu тьнзнс снллой it э шэ фшo1nssшфn snщ цве посероь аолрауй и хоспауй еа ао рузмелероов ьадзы намлой р питсо млгяиж чозанмохауж лнаоауж чус тмна ан бебопоаио хивосой ',
 ' ботле аее сик йи увозок дид зя тпитслнр н тбодоеа м зял ечелн зя коу тдийись себевь уот

In [11]:
BATCH_SIZE = [100, 800, 1500]

def decoding_accuracy(y_true, y_pred):
    return (np.array(list(y_pred)) == np.array(list(y_true))).mean()

def print_accuracy(test1, decoded1, test2, decoded2, test3, decoded3):
    for i, (real, decoded) in enumerate([(test1, decoded1), (test2, decoded2), (test3, decoded3)]):
        print("Batch_size:", BATCH_SIZE[i], " Accuracy: ", round(decoding_accuracy(real, decoded), 3))

print_accuracy(test1, decoded1, test2, decoded2, test3, decoded3)

Batch_size: 100  Accuracy:  0.16
Batch_size: 800  Accuracy:  0.234
Batch_size: 1500  Accuracy:  0.391


*Такой результат, конечно, что-то декодирует, но на выходе текст абсолютно не читаем, поэтому нужно придумывать что-то еще*

# 2. Биграммы

Попробуем теперь использовать биграммы

In [12]:
def make_bigrams(text):
    return [text[i:i+2] for i in range(len(text) - 1)]


def bigramm_mapping(encoded, real):  
    mapping_bigrams = {}
    already_mapped = []
    left_undecoded = []
    
    encoded_count = Counter(encoded).most_common()
    real_count = Counter(real).most_common()
    encoded_bigrams_count = Counter(make_bigrams(encoded)).most_common()
    real_bigrams_count = Counter(make_bigrams(real)).most_common()
    
    for enc, _ in encoded_bigrams_count:
        if enc[0] in mapping_bigrams and enc[1] in mapping_bigrams:
            continue
        elif enc[0] in mapping_bigrams:
            mapped = mapping_bigrams[enc[0]]
            ponential_symbols = [rea for rea, _ in real_bigrams_count if rea[0] == mapped and rea[1] not in already_mapped]
            if len(ponential_symbols) >= 1:
                mapping_bigrams[enc[1]] = ponential_symbols[0][1]
                already_mapped.append(ponential_symbols[0][1])
            else:
                if enc[0] not in left_undecoded:
                    left_undecoded.append(enc[1])
        elif enc[1] in mapping_bigrams:
            mapped = mapping_bigrams[enc[1]]
            ponential_symbols = [rea for rea, _ in real_bigrams_count if rea[1] == mapped and rea[0] not in already_mapped]
            if len(ponential_symbols) >= 1:
                mapping_bigrams[enc[0]] = ponential_symbols[0][0]
                already_mapped.append(ponential_symbols[0][0])
            else:
                if enc[0] not in left_undecoded:
                    left_undecoded.append(enc[0])
        else:
            ponential_symbols = [rea for rea, _ in real_bigrams_count if rea[1] not in already_mapped and rea[0] not in already_mapped]
            mapping_bigrams[enc[0]] = ponential_symbols[0][0]
            mapping_bigrams[enc[1]] = ponential_symbols[0][1]
            already_mapped.append(ponential_symbols[0][0])
            already_mapped.append(ponential_symbols[0][1])
            
    encoded_count = [symbol for symbol, count in encoded_count if symbol in left_undecoded]
    for i, encoded in enumerate(encoded_count):
        mapping_bigrams[encoded] = real_count[i][0]
    
    return mapping_bigrams

mapping_enc_dec1 = bigramm_mapping(encoded_test1, train1)
mapping_enc_dec2 = bigramm_mapping(encoded_test2, train2)
mapping_enc_dec3 = bigramm_mapping(encoded_test3, train3)
decoded1 = decode_text(encoded_test1, mapping_enc_dec1)
decoded2 = decode_text(encoded_test2, mapping_enc_dec2)
decoded3 = decode_text(encoded_test3, mapping_enc_dec3)

In [13]:
decoded1, decoded2, decoded3

('о наяоаоказ ипярязккуамогиеатопркназадьшскапрыйьлавкиотпкжо лартоко лав сашптоаотахяхкдкоькажьукскна',
 'гыьлаокастнхупыпонгчипзнпыпонаяпстпнаогонашов ткойигкове пуптонтбпо п иавсрсгоысавнптоп нпвангтотыпоыплрчтчнедоялтш иавстнгмоигдарозгжнхого япупмнпто тктмнпто ча игтовосе едоыпладоялтш иавсрсг хоткьопноьзтона сазшас ройигко ча игткоупышаовшльыорвсрс рокастнхугмонаяiстпно о впгкобтжьча инекопылангчтннекого ча исгвекопионт ча игрошльыгдовжысршпкогоначгнасг хо пкнтнгрокьугогоипсхупонтбпопбтóаспоь япуптнгтоуоьильов токтчиангро ктъасг хого сгсг ховодап огоклауобт яакри иваогожабвтнгроупиплетоыплажшповтлпринттояпокнтнгщо акпыпосаллтрошпуиплаонаяпстпнаошпсзнеобесголажлтъгих ро ктлихщочтковежшплпвстнгткоэоюоцо фоо nюцофюеaю uоюцоаeseю uо уажасосаллтмоesофоюфоеюэн877юе8о78оойипочтспвтуонтлвнемогозтсчнемопнонтовежшплпвттиоунржхоаншлтмовочг стошльыгдобтжнаштзнедолантнедобесо шанонаояпятчтнгтозгитстмо',
 'ознвуто ттоедрождоаынмнроидиомбовчдвеугсоговзнинт опомбуотэтугомборнаовидждекоетзтыкоанв

In [14]:
print_accuracy(test1, decoded1, test2, decoded2, test3, decoded3)

Batch_size: 100  Accuracy:  0.06
Batch_size: 800  Accuracy:  0.191
Batch_size: 1500  Accuracy:  0.027


*Все еще совсем грустно*

# 3. MCMC-сэмплирование

Попробуем воспользоваться MCMC-сэмплированием для улучшения результатов. А именно, для какого-то случайного декодирования будем случайным образом совершать перестановку двух символов.
После этого, если логарифм правдоподобия дешифровки увеличился, одобряем данную перестановку. Если же уменьшился, то сравним отношение нового правдоподобия к старому со случайным числом от 0 до 1 и одобрим таким образом только часть перестановок.

In [15]:
def random_swap(decoded_symbols):
    new_decode = decoded_symbols.copy()
    left_id, right_id = np.random.choice(range(0, len(decoded_symbols)), size=2, replace=False)
    new_decode[left_id], new_decode[right_id] = new_decode[right_id], new_decode[left_id]
    
    return new_decode


def sum_probs(bigrams, mapa, log_likelyhood, symbol_ids):
    return sum([count * log_likelyhood[symbol_ids[mapa[bigram[0]]], 
                        symbol_ids[mapa[bigram[1]]]] for bigram, count in bigrams]) if bigrams else 0

def mcmc_mapping(encoded, real, n_epoches=10, n_iter=1000):
    probs = []
    new_maps = []
    
    encoded_bigrams = Counter(make_bigrams(encoded)).most_common()
    decoded_bigrams = Counter(make_bigrams(real)).most_common()
    encoded_chars = list(set(encoded))
    decoded_chars = list(set(real))

    
    # Computing bigrams log-likelyhoods
    likelyhood = np.ones((len(decoded_chars), len(decoded_chars)))
    symbol_ids = {symb: i for i, symb in enumerate(decoded_chars)}
    for bigram, count in decoded_bigrams:
        likelyhood[symbol_ids[bigram[0]], symbol_ids[bigram[1]]] += count
    likelyhood /= likelyhood.sum()
    log_likelyhood = np.log(likelyhood)
    
    
    # Sampling
    for _ in range(n_epoches):
        decoded_chars = list(set(real))
        start_map = dict(zip(encoded_chars, decoded_chars))
        prob = sum_probs(encoded_bigrams, start_map, log_likelyhood, symbol_ids)
        
        # Start sampling
        for i in range(n_iter):
            # Swaping random chars
            new_decoded_chars = random_swap(decoded_chars)
            new_map = dict(zip(encoded_chars, new_decoded_chars))
            # Calculating sum log_likelyhood over new (swapped) decoding
            new_prob = sum_probs(encoded_bigrams, new_map, log_likelyhood, symbol_ids)
            
            # If it's better, approve it
            if new_prob > prob:
                decoded_chars = new_decoded_chars
                prob = new_prob
                start_map = new_map
            else:
                # If not, approve it sometimes
                if np.exp(new_prob - prob) > np.random.rand():
                    decoded_chars = new_decoded_chars
                    prob = new_prob
                    start_map = new_map
        
        # Saving epoch results
        probs.append(prob)
        new_maps.append(start_map)
    
    # Get best epoch
    best_index = np.argmax(probs)
    
    return new_maps[best_index]


mapping_enc_dec1 = mcmc_mapping(encoded_test1, train1, 10, 3000)
mapping_enc_dec2 = mcmc_mapping(encoded_test2, train2, 10, 3000)
mapping_enc_dec3 = mcmc_mapping(encoded_test3, train3, 10, 3000)
decoded1 = decode_text(encoded_test1, mapping_enc_dec1)
decoded2 = decode_text(encoded_test2, mapping_enc_dec2)
decoded3 = decode_text(encoded_test3, mapping_enc_dec3)

In [16]:
print_accuracy(test1, decoded1, test2, decoded2, test3, decoded3)

Batch_size: 100  Accuracy:  0.32
Batch_size: 800  Accuracy:  0.39
Batch_size: 1500  Accuracy:  0.277


*Стало немного лучше, но это не то, чего мы ждем*

In [17]:
mapping_enc_dec1 = mcmc_mapping(encoded_test1, train1, 10, 30000)
mapping_enc_dec2 = mcmc_mapping(encoded_test2, train2, 10, 30000)
mapping_enc_dec3 = mcmc_mapping(encoded_test3, train3, 10, 30000)
decoded1 = decode_text(encoded_test1, mapping_enc_dec1)
decoded2 = decode_text(encoded_test2, mapping_enc_dec2)
decoded3 = decode_text(encoded_test3, mapping_enc_dec3)

In [18]:
print_accuracy(test1, decoded1, test2, decoded2, test3, decoded3)

Batch_size: 100  Accuracy:  0.27
Batch_size: 800  Accuracy:  0.95
Batch_size: 1500  Accuracy:  0.973


In [19]:
decoded1, decoded2, decoded3

('тай ут то васкурувоол этысь иткрой в дезно крющея постикочтая ритотая пан зкит ти гугодотео челоной ',
 'игура маленького ничтожного наполеона и над всем этим высокое небо составляли главное основание его горячечных представлений тихая жизнь и спокойное семейное счастие в лысых горах представлялись ему он уже наслаждался этим счастием когда вдруг являлся маленький напшлеон с своим безучастным ограниченным и счастливым от несчастия других взглядом и начинались сомнения муки и только небо обецало успокоение к утру все мечтания смещались и слились в хаос и мрак беспамятства и забвения которые гораздо вероятнее по мнению самого ларрея доктора наполеона должны были разрещиться смертью чем выздоровлением m eur ns under sechent er avivent сказал ларрей vi s es cembolleco lou это человек нервный и желчный он не выздоровеет князь андрей в числе других безнадежных раненых был сдан на попечение жителей ',
 ' после нее там за гробом как бы счастлив и спокоен я был ежели бы мог сказать теперь гос

*А вот теперь уже похоже на правду*

# 4. Расшифруем сообщение

In [20]:
# encoded_ = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨ" \
#            "ჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣს" \
#            "ჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

encoded_ = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

expected_output = "если вы видите нормальный или почти нормальный текст у этого сообшения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

In [22]:
mapping_enc_dec_ = mcmc_mapping(encoded_, corpus, 30, 30000)
decoded_ = decode_text(encoded_, mapping_enc_dec_)

In [23]:
decoded_

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

In [24]:
print("Accuracy: ", round(decoding_accuracy(expected_output, decoded_), 3))

Accuracy:  0.957


*Как видно, результат вполне читаемый*